# Value-at-Risk for Stocks: Historical Simulation

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling
import matplotlib.pyplot as plt # Plotting
from scipy.stats import norm # statistical analysis
from statistics import NormalDist # statistical analysis

%matplotlib inline


## Question on Single-Asset Portfolio:
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares. Determine the portfolio's one-day 99% VaR using historical simulation approach.
 


In [2]:
def get_return(df, d):
    """
    df is the original df
    
    appends returns series to df
    """
    df["previous"] = df["close"].shift(-d)
    df["return"] = np.log(df["close"]/df["previous"])
    return df 

In [3]:
def get_change_in_value_df(df, N):
    """
    df is the output of get_returns_df 
    
    appends change in portfolio value series to df
    """
    S0 = df.loc[0, "close"]
    df["change_in_value"] = N * S0 * df["return"]
    return df

In [4]:
def get_kth_percentile_discrete(df, d, p):
    """
    this returns the (1-p/100)th percentile of the ordered array of historical changes in portfolio values
    """
    M = len(df["change_in_value"])-d
    alpha = p/100
    k = int(np.floor((1-alpha)*M))
    var = abs(df["change_in_value"].nsmallest(k).values[-1])
    
    return var

In [5]:
def d_day_p_percent_VaR_single_stock_historical(
    df, N, d, p
):
    """Returns the d-day p% VaR of a single stock using Historical Simulation Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    p : float
        the value to be used in calculting the p% VaR (e.g. 99, 95, 97.5)
    
    Returns
    -------
    float (2 decimal places)
        d-day p% VaR of a single stock using Historical Simulation Approach
    """
    
    # step 1: generate your historical returns
    df = get_return(df, d)
    # Step 2: get your historical changes in portfolio values
    df = get_change_in_value_df(df, N)
    # Step 3: get the (1-p/100)th percentile of the ordered array of historical changes in portfolio values
    var = get_kth_percentile_discrete(df, d, p)
    
    return round(var, 2)

In [14]:
TEL_df = pd.read_csv("../data/TEL_2018.csv")

In [7]:
d_day_p_percent_VaR_single_stock_historical(df=TEL_df, N=700, d=1, p=99)

60730.66

## Question on N-Asset Portfolio:
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares, 300 Ayala Corporation (AC) shares, and 500 URC shares. What is the portfolio's one-day 99% VaR using historical approach?

In [22]:
TEL_df = pd.read_csv("../data/TEL_2018.csv")
AC_df = pd.read_csv("../data/AC_2018.csv")
URC_df = pd.read_csv("../data/URC_2018.csv")

In [11]:
TEL_df[["dt", "close"]]

,dt,close
0,2/23/18,1488.74
1,2/22/18,1510.86
2,2/21/18,1513.72
3,2/20/18,1536.65
4,2/16/18,1476.37
...,...,...
243,3/2/17,1430.29
244,3/1/17,1408.94
245,2/28/17,1394.35
246,2/27/17,1374.93


In [19]:
def get_portfolio_value_df(df_lst, N_lst):
    
    portval_df = pd.DataFrame(0, index=df_lst[0].index, columns=["close"])
    
    for df, N in zip(df_lst, N_lst):
        stock_value_series = df["close"] * N
        portval_df["close"] = portval_df["close"] + stock_value_series  
    
    return portval_df 

In [23]:
# TEST

# get_portfolio_value_df(df_lst=[TEL_df], N_lst=[1000])

#N = 1000
#TEL_df["close"]*N

get_portfolio_value_df(df_lst=[TEL_df, AC_df, URC_df], N_lst=[700,300,500])

,close
0,1389705.0
1,1405463.0
2,1412221.0
3,1428132.0
4,1387394.0
...,...
243,1315199.0
244,1299949.0
245,1288700.0
246,1273753.0


$\Delta P_j = P_0 e^{R_j^P} - P_0 = P_0 (e^{R_j^P} -1)$

In [25]:
def get_change_in_value_df_alternative(df):
    """
    df is the output of get_returns_df 
    
    appends change in value series (using alternative approach/exponential function) to df
    """
    P0 = df.loc[0, "close"]
    df["change_in_value"] = P0 * (np.exp(df["return"]) - 1)
    
    return df

In [28]:
def d_day_p_percent_VaR_portfolio_historical_alternative(
    df_lst, N_lst, d, p
):
    """Returns the d-day p% VaR of a portfolio using the Alternative Historical Simulation Approach.
    
    Parameters
    ----------
    df_lst : list of pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row
        asssumes that all dfs have the same ordered list of dates
        there is one df per stock in the portfolio
    N_lst : list (of int)
        N_i = number of shares for ith stock (order should be the same as the order in df_lst)
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    p : float
        the value to be used in calculting the p% VaR (e.g. 99, 95, 97.5)
    
    Returns
    -------
    float (2 decimal places)
        d-day p% VaR of a portfolio using the Alternative Historical Simulation Approach
    """
    
    # Step 1: generate the list of portfolio values
    df = get_portfolio_value_df(df_lst, N_lst)
    # step 2: get the returns of this list of portfolio values
    df = get_return(df, d)
    # Step 3: get the list of M scenarios for your change in portfolio value
    df = get_change_in_value_df_alternative(df)
    # Step 4: calculate the VaR by getting the (1-p/100)th percentile of the ordered list of M scenarios for your change in portfolio value
    var = get_kth_percentile_discrete(df, d, p)
                                            
    return round(var, 2)

In [29]:
d_day_p_percent_VaR_portfolio_historical_alternative(
    df_lst=[TEL_df, AC_df, URC_df], 
    N_lst=[700,300,500], 
    d=1, 
    p=99
)

58888.51